# Kerr Angle Conversions Example Notebook

## Loading the package

In [1]:
import os
import sys

from time import perf_counter

# Need to point to where the package is installed
# It will always be in the build directory, but the next folder name might change on your machine
sys.path.append(os.path.abspath('../build/lib.macosx-11.0-arm64-cpython-39'))
import kerrangleconversions as kac

import numpy as np

## Example Conversion

In [2]:
num = 10
a = 0.9
p = 10
e = 0.3
x = 0.8
psi = 1.2
chi = 0.5
phi = 10

In [3]:
time = perf_counter()
(Phi_r,Phi_theta,Phi_phi) = kac.pyDarwinPhasesToBoyerLindquistPhases(a,p,e,x,psi,chi,phi)
total_time = (perf_counter() - time) *1000
print("Time taken: %.3f ms" % total_time)
(Phi_r,Phi_theta,Phi_phi)

Time taken: 0.143 ms


(0.7656854348230792, -0.0866817374910737, 9.264252315901594)

In [4]:
time = perf_counter()
(psi2,chi2,phi2) = kac.pyBoyerLindquistPhasesToDarwinPhases(a,p,e,x,Phi_r,Phi_theta,Phi_phi)
total_time = (perf_counter() - time) *1000
print("Time taken: %.3f ms" % total_time)
(psi2,chi2,phi2)

Time taken: 0.248 ms


(1.1999999999998807, 0.4999999999998478, 9.999999999999941)

In [5]:
(psi-psi2,chi-chi2,phi-phi2)

(1.192379528447418e-13, 1.5221157667610896e-13, 5.861977570020827e-14)

## Trajectory Evolution

In [6]:
kac.pyRadialDarwinFrequency(a,p,e,x,psi,chi)

0.02764311159588846

In [7]:
kac.pyPolarDarwinFrequency(a,p,e,x,psi,chi)

0.035469176998042654

In [8]:
kac.pyAzimuthalDarwinFrequency(a,p,e,x,psi,chi)

0.04126053109895384

In [9]:
kac.pyRadialFrequency(a,p,e,x)

0.020809663097485945

In [10]:
kac.pyPolarFrequency(a,p,e,x)

0.026436776509709877

In [11]:
kac.pyAzimuthalFrequency(a,p,e,x)

0.027775598854310045